### only install once

In [ ]:
%%bash
## only install once
sudo apt-get install awscli -y 


### please fill out the following AWS credential so we can get the ECR docker container

In [ ]:
%env AWS_ACCESS_KEY_ID=""
%env  AWS_SECRET_ACCESS_KEY=""
%env  AWS_SESSION_TOKEN=""
%env  AWS_DEFAULT_REGION=us-east-1
%env  aws_account_id=
%env  repository_name=ros-humble-greengrass-minipupper
%env  tag=latest


In [ ]:
%%bash
# login
aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin ${aws_account_id}.dkr.ecr.us-east-1.amazonaws.com

echo ${aws_account_id}.dkr.ecr.${AWS_DEFAULT_REGION}.amazonaws.com/${repository_name}:${tag}
# pull ecr image
docker pull ${aws_account_id}.dkr.ecr.${AWS_DEFAULT_REGION}.amazonaws.com/${repository_name}:${tag}

# retag the docker image
docker tag ${aws_account_id}.dkr.ecr.${AWS_DEFAULT_REGION}.amazonaws.com/${repository_name}:${tag} ${repository_name}:${tag} 

In [ ]:
!docker-compose up